# Estimating Telecommute Frequency Model

This notebook re-estimates ActivitySim telecommute frequency model in Larch. 

# Load libraries

In [2]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd
import numpy as np
from larch import P, X
import matplotlib.pyplot as plt

C:\Users\joel.freedman\AppData\Local\mambaforge\envs\asim_dev\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\joel.freedman\AppData\Local\mambaforge\envs\asim_dev\lib\site-packages\sharrow\accessors.py:23: AccessorRegistrationWarning: registration of accessor <function register_dataarray_method.<locals>.wrapper at 0x0000026E79336E50> under name 'to_zarr' for type <class 'xarray.core.dataarray.DataArray'> is overriding a preexisting attribute with the same name.
  return xr.register_dataarray_accessor(func.__name__)(wrapper)


The directory with the estimation data bundles for telecommute frequency.

In [3]:
os.chdir('/projects/SANDAG/2017 On-Call Modeling Services/Area B/TO 05 - ABM3/estimation/output/estimation_data_bundle/telecommute_frequency')

# Drop duplicate person records

In [4]:
alts_combined_data = pd.read_csv("telecommute_frequency_values_combined_orig.csv", dtype={'household_id': np.int64},low_memory=False)
#household_data = pd.read_csv("../override_households.csv",dtype={'household_id': np.int64})
#person_data = pd.read_csv("../override_persons.csv")
alts_combined_data.shape

(38303, 302)

In [5]:
#merge alts_combined data with person file
#alts_combined_per_data = pd.merge(alts_combined_data, person_data[['person_id','PNUM']], on=["person_id"], how='left')
#alts_combined_per_data.shape

In [6]:
#merge alts_combined data (with person file) with household file
#alts_combined_hh_data = pd.merge(alts_combined_data, household_data[['household_id','HH_ID']], on=["household_id"], how='left')
#alts_combined_hh_data.shape

In [7]:
#drop duplicate person records and merged columns
alts_combined_data = alts_combined_data.sort_values(by=['override_choice'])
alts_combined_new_data = alts_combined_data.drop_duplicates(subset=['HH_ID','PNUM'])
alts_combined_new_data = alts_combined_new_data[alts_combined_new_data['is_worker']==True]
#alts_combined_new_data = alts_combined_new_data.drop(columns=['household_id','PNUM','HH_ID'])
alts_combined_new_data.shape

(7309, 302)

In [8]:
#recalculate util_2016 based on survey year
alts_combined_new_data['util_2016'] = np.where(alts_combined_new_data['survey_year']==2016,1,0)
alts_combined_new_data['util_part_time'] = np.where(alts_combined_new_data['is_parttime_worker']==True,1,0)
alts_combined_new_data['util_accomodation'] = np.where(alts_combined_new_data['industry']=='accomodation',1,0)
alts_combined_new_data['util_agriculture'] = np.where(alts_combined_new_data['industry']=='agriculture',1,0)
alts_combined_new_data['util_business_srv'] = np.where(alts_combined_new_data['industry']=='business_srv',1,0)
alts_combined_new_data['util_construction'] = np.where(alts_combined_new_data['industry']=='construction',1,0)
alts_combined_new_data['util_education'] = np.where(alts_combined_new_data['industry']=='education',1,0)
alts_combined_new_data['util_entertainment'] = np.where(alts_combined_new_data['industry']=='entertainment',1,0)
alts_combined_new_data['util_food_srv'] = np.where(alts_combined_new_data['industry']=='food_srv',1,0)
alts_combined_new_data['util_government'] = np.where(alts_combined_new_data['industry']=='government',1,0)
alts_combined_new_data['util_healthcare'] = np.where(alts_combined_new_data['industry']=='healthcare',1,0)
alts_combined_new_data['util_manufacturing'] = np.where(alts_combined_new_data['industry']=='manufacturing',1,0)
alts_combined_new_data['util_mgmt_srv'] = np.where(alts_combined_new_data['industry']=='mgmt_srv',1,0)
alts_combined_new_data['util_military'] = np.where(alts_combined_new_data['industry']=='military',1,0)
alts_combined_new_data['util_retail'] = np.where(alts_combined_new_data['industry']=='retail',1,0)
alts_combined_new_data['util_asc'] = 1


In [9]:
#write data to file
if os.path.exists("telecommute_frequency_values_combined_orig.csv")!=True:
  shutil.copy2('telecommute_frequency_values_combined.csv','telecommute_frequency_values_combined_orig.csv')
alts_combined_new_data.to_csv("telecommute_frequency_values_combined.csv", index=False)

# Load data and prep model for estimation

In [10]:
os.chdir('/projects/SANDAG/2017 On-Call Modeling Services/Area B/TO 05 - ABM3/estimation')
modelname = "telecommute_frequency"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

c:\development\projects\sandag_activitysim\activitysim\estimation\larch\simple_simulate.py:66: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)


# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [11]:
data.coefficients

,value,constrain
coefficient_name,,
coef_Services_1day,0,F
coef_SalesOffice_1day,0,F
coef_ResourceConstruct_1day,0,F
coef_TransportMat_1day,0,F
coef_HasChildren0to5_1day,0,T
...,...,...
coef_military_4day,0,F
coef_retail_4day,0,F
asc_1day,0,F


#### Utility specification

In [12]:
data.spec

,Label,Description,Expression,No_Telecommute,1_day_week,2_3_days_week,4_days_week
0,util_HasChildren0to5,Has children 0 to 5 years old,@df.num_young_children>0,NaN,coef_HasChildren0to5_1day,coef_HasChildren0to5_234day,coef_HasChildren0to5_234day
1,util_HasChildren6to12,Has children 6 to 12 years old,@df.num_children_6_to_12>0,NaN,coef_HasChildren6to12_1day,coef_HasChildren6to12_23day,coef_HasChildren6to12_4day
2,util_OneAdultInHH,One adult in hh,@df.num_adults==1,NaN,coef_OneAdultInHH_1day,coef_OneAdultInHH_23day,coef_OneAdultInHH_4day
3,util_Female,female,@df.female,NaN,coef_Female_1234day,coef_Female_1234day,coef_Female_1234day
4,util_PartTimeWorker,Part-time worker,@df.pemploy==2,NaN,coef_PartTimeWorker_1234day,coef_PartTimeWorker_1234day,coef_PartTimeWorker_1234day
5,util_Income60to100k,Income 60-100k,"@df.income.between(60000, 100000)",NaN,coef_Income60to100k_1day,coef_Income60to100k_23day,coef_Income60to100k_4day
6,util_Income100to150k,Income 100-150k,"@df.income.between(100000, 150000)",NaN,coef_Income100to150k_1day,coef_Income100to150k_234day,coef_Income100to150k_234day
7,util_Income150kplus,Income 150k+,@df.income > 150000,NaN,coef_Income150kplus_1day,coef_Income150kplus_23day,coef_Income150kplus_4day
8,util_0Autos,0 Autos,@df.auto_ownership==0,NaN,coef_0Autos_1day,coef_0Autos_234day,coef_0Autos_234day
9,util_1Auto,1 Auto,@df.auto_ownership==1,NaN,coef_1Auto_1day,coef_1Auto_234day,coef_1Auto_234day


## Explore data

In [13]:
data.chooser_data

,person_id,model_choice,override_choice,util_HasChildren0to5,util_HasChildren6to12,util_OneAdultInHH,util_2plusAdultsInHH,util_Female,util_PartTimeWorker,util_CollegeStudent,...,util_entertainment,util_food_srv,util_government,util_healthcare,util_manufacturing,util_mgmt_srv,util_military,util_retail,util_asc,override_choice_code
household_id,,,,,,,,,,,,,,,,,,,,,
33961,67009,1_day_week,1_day_week,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,2
29671,58590,4_days_week,1_day_week,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,2
42929,84716,2_3_days_week,1_day_week,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,2
29668,58582,2_3_days_week,1_day_week,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,2
7075,13954,2_3_days_week,1_day_week,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16963,33499,1_day_week,No_Telecommute,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,1
16468,32476,2_3_days_week,No_Telecommute,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,1
17900,35345,4_days_week,No_Telecommute,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,1


In [14]:
pd.crosstab(data.chooser_data.override_choice, data.chooser_data.survey_year, margins=True)

survey_year,2016,2022,All
override_choice,,,
1_day_week,737,224,961
2_3_days_week,283,210,493
4_days_week,35,312,347
No_Telecommute,4317,1191,5508
All,5372,1937,7309


# Set Coefficients

In [15]:
dir(model)
print(model.utility_co)

DictOfLinearFunction_C({1: <Empty LinearFunction_C>, 2:   P.coef_HasChildren0to5_1day * X.util_HasChildren0to5
+ P.coef_HasChildren6to12_1day * X.util_HasChildren6to12
+ P.coef_OneAdultInHH_1day * X.util_OneAdultInHH
+ P.coef_Female_1234day * X.util_Female
+ P.coef_PartTimeWorker_1234day * X.util_PartTimeWorker
+ P.coef_Income60to100k_1day * X.util_Income60to100k
+ P.coef_Income100to150k_1day * X.util_Income100to150k
+ P.coef_Income150kplus_1day * X.util_Income150kplus
+ P.coef_0Autos_1day * X.util_0Autos
+ P.coef_1Auto_1day * X.util_1Auto
+ P.coef_3plusAutos_1day * X.util_3plusAutos
+ P.coef_DistanceToWork_1day * X.util_DistanceToWork
+ P.coef_calib_2020_1day * X.util_calib_2020
+ P.coef_calib_2025_1day * X.util_calib_2025
+ P.coef_calib_2035_1day * X.util_calib_2035
+ P.coef_calib_2050_1day * X.util_calib_2050
+ P.coef_2016_1day * X.util_2016
+ P.coef_accomodation_1234day * X.util_accomodation
+ P.coef_agriculture_1234day * X.util_agriculture
+ P.coef_business_srv_1day * X.util_busin

In [16]:
#model.utility_co = {0: P.coef_dist_to_nearest_ext_station * X.util_dist_to_nearest_ext_station
#+ P.coef_size_of_nearest_ext_station * X.util_size_of_nearest_ext_station
#+ P.coef_part_time * X.parttime
#+ P.coef_agriculture * X.agriculture
#+ P.coef_business_srv * X.business_srv
#+ P.coef_construction * X.construction
#+ P.coef_education * X.education
#+ P.coef_entertainment * X.entertainment
#+ P.coef_food_srv * X.food_srv                   
#+ P.coef_government * X.government
#+ P.coef_healthcare * X.healthcare                   
#+ P.coef_manufacturing * X.manufacturing
#+ P.coef_mgmt_srv * X.mgmt_srv
#+ P.coef_military * X.military
#+ P.coef_retail * X.retail    
#+ P.coef_inc_lt15 * X("income<15000")
#+ P.coef_inc_15_25 * X("(income>=15000) * (income<25000)") 
#+ P.coef_inc_25_50 * X("(income>=25000) * (income<50000)") 
#+ P.coef_inc_100_150 * X.income_100_150 
#+ P.coef_inc_150_250 * X("(income>=150000) * (income<250000)")  
#+ P.coef_inc_250plus * X("income>=250000")
#+ P.asc_external_2016 * X.year_2016         
#+ P.coef_dist_lt_2p5 * X("util_dist_to_nearest_ext_station<2.5")                    
#+ P.asc_external_worker * X.util_asc_placeholder, 1: 0}

# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [17]:
model.load_data()
#model.doctor(repair_ch_av="-")

req_data does not request avail_ca or avail_co but it is set and being provided
converting data_co to <class 'numpy.float64'>


In [18]:
model.maximize_loglike(method="SLSQP", options={"maxiter": 1000})


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
asc_1day,-1.821844,0.0,0.0,NaN,NaN,0,,-1.821844
asc_23day,-1.768458,0.0,0.0,NaN,NaN,0,,-1.768458
asc_4day,-1.372632,0.0,0.0,NaN,NaN,0,,-1.372632
coef_0Autos_1day,0.000000,0.0,0.0,0.0,0.0,1,,0.000000
coef_0Autos_234day,0.000000,0.0,0.0,0.0,0.0,1,,0.000000
...,...,...,...,...,...,...,...,...
coef_military_1day,0.000000,0.0,0.0,0.0,0.0,1,,0.000000
coef_military_234day,-0.512647,0.0,0.0,NaN,NaN,0,,-0.512647
coef_retail_1day,-0.832034,0.0,0.0,NaN,NaN,0,,-0.832034
coef_retail_23day,-0.757629,0.0,0.0,NaN,NaN,0,,-0.757629


┣                x: asc_1day               -1.821844
┃                   asc_23day              -1.768458
┃                   asc_4day               -1.372632
┃                   coef_0Autos_1day        0.000000
┃                   coef_0Autos_234day      0.000000
┃                                             ...   
┃                   coef_military_1day      0.000000
┃                   coef_military_234day   -0.512647
┃                   coef_retail_1day       -0.832034
┃                   coef_retail_23day      -0.757629
┃                   coef_retail_4day       -2.233899
┃                   Length: 70, dtype: float64
┣          loglike: -5283.181318490329
┣        d_loglike: asc_1day                0.017476
┃                   asc_23day              -0.006657
┃                   asc_4day               -0.004857
┃                   coef_0Autos_1day        0.000000
┃                   coef_0Autos_234day      0.000000
┃                                             ...   
┃                   coef_military_1day      0.000000
┃                   coef_military_234day   -0.000913
┃                   coef_retail_1day        0.000834
┃                   coef_retail_23day      -0.000452
┃                   coef_retail_4day       -0.000621
┃                   Length: 70, dtype: float64
┣              nit: 41
┣             nfev: 110
┣             njev: 41
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=12, microseconds=588110)
┣           method: 'SLSQP'
┣          n_cases: 7309
┣ iteration_number: 41
┣          logloss: 0.722832305170383

### Estimated coefficients

In [19]:
model.calculate_parameter_covariance()
result_dir='/projects/SANDAG/2017 On-Call Modeling Services/Area B/TO 05 - ABM3/estimation/'
model.to_xlsx(
        result_dir+"telecommute_frequency_004.xlsx", 
        data_statistics=True,
    )

C:\Users\joel.freedman\AppData\Local\mambaforge\envs\asim_dev\lib\site-packages\larch\util\excel.py:523: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  xl = ExcelWriter(filename, engine='xlsxwriter_larch', model=model, **kwargs)


# Output Estimation Results

In [20]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_004.csv",
);

In [21]:
#larch.__version__

In [22]:
#result_dir

### Write the model estimation report, including coefficient t-statistic and log likelihood

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [23]:
#pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")